In [1]:
def load_data(path1 , path2):
    '''pass the two paths'''
    import warnings
    warnings.filterwarnings("ignore")
    import pandas as pd
    df1 = pd.read_json(path1,lines=True)
    df2 =pd.read_json(path2,lines = True)
    df = pd.merge(df1,df2,on = 'asin')
    df.drop(['imageURLHighRes','imageURL','vote','image','tech2','tech1','similar_item','date','category','feature','main_cat','description','fit'],axis = 1 , inplace = True)
    df.dropna(inplace = True)
    import numpy as np 
    df.index = np.arange(140833)  # resest index 
    return df

In [2]:
def visual(df,col1,col2):
    import matplotlib.pyplot as plt 
    import seaborn as sns
    plt.subplot(1,2,1)
    sns.countplot(df[col1]);
    plt.title("overall ration(1-5)",size = 20,c = 'red');
    plt.xlabel("Ratings",size = 15);
    plt.subplot(1,2,2)
    sns.countplot(df[col2]);
    plt.title("reviews verify",size = 20,c = 'red');
    plt.xlabel("verification",size = 15);
    plt.tight_layout();

In [3]:
def info(df):
    print(df.shape)
    print("\n##########################################################\n")
    print(df.dtypes)
    print("\n##########################################################\n")
    print(df.info())
    print("\n##########################################################\n")
    print(df.describe())
    print("\n##########################################################\n")
    print(df.isnull().sum())

In [5]:
def price_cleaning(col):
    import re 
    text = re.sub("\$",'',col)
    text = re.sub("\,","",text)        
    text = text.strip()
    text = float(text)
    return text 

In [8]:
def price_visual(df,col):
    import matplotlib.pyplot as plt 
    import seaborn as sns
    plt.figure(figsize=(8,3))
    plt.subplot(1,2,1)
    sns.boxenplot(df[col]);
    plt.subplot(1,2,2)
    sns.distplot(df[col]);
    plt.tight_layout();

In [9]:
def rank_cleaning(col):
    import re 
    text = col.strip()
    text = re.sub('\,','',text)
    text = re.sub('\;',',',text)
    text = re.sub(r"[^a-zA-Z\&\,\s+]","",text)
    text = text.strip()
    text = text.lstrip()[2:]
    return text

In [11]:
def rank_category_clean(df1):
    import numpy as np 
    import matplotlib.pyplot as plt 
    import seaborn as sns
    import pandas as pd
    index = np.where(df1['rank_category'] == ' Books')
    index = index[0]
    df1 = df1.drop(index,axis = 0)
    sns.countplot(df1['rank_category']);
    print(df1['rank_category'].value_counts())
    print("################################################################")
    return df1

In [13]:
def rank_num_cleaning(col):
    import re 
    text = re.sub(r"[^0-9]","",col)
    text = re.sub('\,','',text)
    text = text.strip()
    text = int(text)
    return text

In [14]:
def rank_num_visual(df1):
    import numpy as np 
    import matplotlib.pyplot as plt 
    import seaborn as sns
    import pandas as pd
    sns.distplot(df1['rank_num']);
    df1.drop(['rank','Unnamed: 0'],axis = 1,inplace = True)
    df1.index = np.arange(91700)
    return df1

In [15]:
def review_time(col):
    import re
    text = re.sub("\,","",col)
    text = re.sub("\s+","-",text)
    text = text.strip()
    return text

In [17]:
def review_time_clean(df1,col):
    import numpy as np 
    import matplotlib.pyplot as plt 
    import seaborn as sns
    import pandas as pd
    df1['reviewTime'] = pd.to_datetime(df1[col])
    df1.sort_values(by = ['reviewTime'],inplace= True)
    df1.index = np.arange(91700)
    return df1

In [19]:
def style_clean(col):
    import re
    text = col.lstrip()[12:]
    text = re.sub("[\},\']","",text)
    text = text.strip()
    return text

In [20]:
def style_visual(df1):
    import numpy as np 
    import matplotlib.pyplot as plt 
    import seaborn as sns
    import pandas as pd
    sns.countplot(df1['style']);
    plt.xticks(rotation = 90);
    print(df1['style'].value_counts())
    print("###################################################################")
    df1.dropna(inplace = True)
    print(df1.shape)
    print("###################################################################")
    return df1

In [21]:
def uniexreviewtime(col):
    import datetime
    text = datetime.datetime.fromtimestamp(col).isoformat()
    return text

In [23]:
def details_label(str1):
    import re
    val = str1.find("Label")
    if val<0:
        return 'NA'
    elif val>=0:
        str2 = str1.split("Label:",1)[1]
        str3 =  str2.split(',')[0]
        str3 = re.sub(r"[^a-zA-Z]"," ",str3)
        str3 = re.sub("\s+"," ",str3)
        str3 = str3.strip()
        return str3

In [24]:
def label_clean(df1):
    import pandas as pd
    import numpy as np 
    df1.index = np.arange(82390)
    index = np.where(df1.Label == 'IMPORTS')
    for i in index:
        df1.loc[i,'Label'] = 'Import' 
    index = np.where(df1.Label == 'Universal Japan')
    for i in index:
        df1.loc[i,'Label'] = 'UNIVERSAL MUSIC JAPAN'
    print(df1.Label.value_counts())
    df1.drop(['details','also_view','also_buy'],axis = 1,inplace = True)
    return df1

In [2]:
def reviews_cleaning(text):
    import re 
    # Remove all non-letters and non-spaces except for hyphens and digits
    text = re.sub("[^0-9A-Za-z\- ]+", " ", text)
    # Remove all numbers except those attached to a word
    text = re.sub("(?<!\w)\d+", "", text)
    # Remove all hyphens except between two words
    text = re.sub("-(?!\w)|(?<!\w)-", "", text)
    # Remove multiple spaces and lowercase everything
    text = " ".join(text.split())
    text = text.lower()
    return text

In [3]:
def tfidf(df,col):
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(stop_words="english")
    # Compute the TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform(df[col])
    review_num=[]
    for i in tfidf_matrix:
        review_num.append(i.toarray().sum())
    return review_num

In [4]:
def label_encoding(df , col):
    from sklearn import preprocessing
    label_encoder = preprocessing.LabelEncoder()  # object is made of method
    df[col]=label_encoder.fit_transform(df[col])  
    return df[col]

In [6]:
def normalization(df):
    import pandas as pd
    from sklearn.preprocessing import MinMaxScaler
    scalar_again = MinMaxScaler()   # call the method
    df_scaled = pd.DataFrame(scalar_again.fit_transform(df),columns=df.columns)
    return df_scaled

In [10]:
def seasonal(df,period):
    import matplotlib.pyplot as plt
    from statsmodels.tsa.seasonal import seasonal_decompose 
    seasonal_decompose(df,period = int(period)).plot();

In [1]:
def clustering(df1 , col1 , col2):
    from sklearn.cluster import KMeans
    import pandas as pd
    import numpy as np 
    x = np.array(df1.loc[:,[col1,col2]]).reshape(-1,2)
    model = KMeans(n_clusters = 3)
    model.fit(x)
    df1['kmean_polarity']=model.labels_
    df1['kmean_polarity'].replace(to_replace=[0,2,1], value=["positive","neutral","negative"],inplace=True)
    import seaborn as sns 
    import matplotlib.pyplot as plt
    sns.scatterplot(x = df1[col1], y = df1[col2] ,  hue = df1['kmean_polarity'],palette='Set1');
    plt.show();

In [1]:
def load_data_musical(path1 , path2):
    '''pass the two paths'''
    import warnings
    warnings.filterwarnings("ignore")
    import pandas as pd
    df1 = pd.read_json(path1,lines=True)
    df2 =pd.read_json(path2,lines = True)
    df = pd.merge(df1,df2,on = 'asin',how = 'inner')
    df.drop(["fit","tech1","also_buy","also_view","similar_item","imageURL","imageURLHighRes","tech2","details","vote","unixReviewTime","image"],axis=1,inplace=True)
    df.dropna(inplace = True)
    len1 = len(df)
    import numpy as np 
    df.index = np.arange(len1)  # resest index 
    return df

In [7]:
def products(data):          #FETCHING PRODUCTS FROM CATEGORY COLUMN
    import re 
    from collections import Counter
    from nltk.tokenize import sent_tokenize, word_tokenize
    y=re.sub("[\'\,\&]","",str(data))
    most_common_list = Counter(word_tokenize(y)).most_common(1)
    return most_common_list 

In [8]:
def product_clean(data):                                           # TO CLEAN THE PRODUCT COLUMNS
    import re 
    t=re.sub("[\[\]\(\)\,\d\']","",str(data))
    
    return t

In [11]:
def replace_products(text):
    import re 
    text=re.sub("(Accessories|Microphones|Recording|amp|Studio|Amplifiers|Keybords|Mixers|Speakers|Lighting|Processors|Electronic|Recorders)","Accessories",str(text))
    text=re.sub("(Guitars|Ukuleles|Violins|Violas|Cellos|Banjos|Mandolins)","String Instruments",text)
    text=re.sub("(Precussion|Drums|Drum)","Precussion Instruments",text)
    text=re.sub("(Effects|Stage|Instruments|DJ|dj|Karaoke|Strobe|and|Instrument|with|AC|Upright|Chimes|French|nylon)","Others",text)
    return text

In [12]:
def style(str1):
    import re 
    str1=str(str1)
    val = str1.find("Style:")
    if val<0:
        return 'NA'
    elif val>=0:
        str2 = str1.split("Style:",1)[1]
        str3 =  str2.split(',')[0]
        str3 = re.sub(r"[^a-zA-Z]"," ",str3)
        str3 = re.sub("\s+"," ",str3)
        str3 = str3.strip()
        return str3

In [13]:
def color(str1):
    import re
    str1=str(str1)
    val = str1.find("Color:")
    if val<0:
        return 'NA'
    elif val>=0:
        str2 = str1.split("Color:",1)[1]
        str3 =  str2.split(',')[0]
        str3 = re.sub(r"[^a-zA-Z]"," ",str3)
        str3 = re.sub("\s+"," ",str3)
        str3 = str3.strip()
        return str3

In [14]:
def format(str1):
    import re 
    str1=str(str1)
    val = str1.find("Format:")
    if val<0:
        return 'NA'
    elif val>=0:
        str2 = str1.split("Format:",1)[1]
        str3 =  str2.split(',')[0]
        str3 = re.sub(r"[^a-zA-Z]"," ",str3)
        str3 = re.sub("\s+"," ",str3)
        str3 = str3.strip()
        return str3

In [16]:
def rank_clean(text):
    import re 
    text=re.sub("[\>#\,\(\)]","",text)
    text=re.search("\d+",text)
    return text

In [17]:
def rank_clean1(text):
    text = text.group()
    return text

In [18]:
def musical(string):     #grouping the categories
    import re 
    string=re.sub(r'(Precussion Others|Keyboards|Percussion)','Percussion Instruments',string)
    string=re.sub(r'(String Others|Others)','String Instruments',string)
    return string 

In [1]:
def clusteringg(df1 , col1 , col2):
    from sklearn.cluster import KMeans
    import pandas as pd
    import numpy as np 
    x = np.array(df1.loc[:,[col1,col2]]).reshape(-1,2)
    model = KMeans(n_clusters = 3)
    model.fit(x)
    df1['kmean_polarity']=model.labels_
    df1['kmean_polarity'].replace(to_replace=[0,2,1], value=["neutral","negative","positive"],inplace=True)
    import seaborn as sns 
    import matplotlib.pyplot as plt
    sns.scatterplot(x = df1[col1], y = df1[col2] ,  hue = df1['kmean_polarity'],palette='Set1');
    plt.show();

In [1]:
def visual(name , df,df1):
    import matplotlib.pyplot as plt 
    plt.figure(figsize=(20,5))
    plt.plot(df,label='Historical Data',linewidth=3)
    plt.plot(df1['overall'],color='orange',label='Predicted Data',linewidth=3)
    plt.title(f'Predictions of {name}')
    plt.legend()
    plt.xlabel("Years")
    plt.ylabel("Ratings");